In [1]:
import networkx as nx
import graphic_generator # El archivo que crea el grafo de los miserables, está incluído en los archivos subidos
import matplotlib.pyplot as plt
from community import community_louvain
import numpy as np
import random

In [4]:
Gd = graphic_generator.les_miserables_graph(directed=True)

# Grafo dirigido
print(f'Tipo de Objeto: {type(Gd)}')
print(f'Número de nodos: {Gd.__len__()}')
print(f'Número de ejes: {Gd.size()}')

nodos = list(Gd.nodes())


Tipo de Objeto: <class 'networkx.classes.digraph.DiGraph'>
Número de nodos: 77
Número de ejes: 254


In [135]:
class Comunidad:
    def __init__(self, nodo_inicial, grafo: nx.Graph):
        self.lista_nodos = [nodo_inicial]
        self.supergrafo = grafo
        self.subgrafo = grafo.subgraph(nodo_inicial)
        self.m = self.supergrafo.size(weight = 'weight')
        self.outer_edges = [arc for arc in self.supergrafo.edges() if arc not in self.subgrafo.edges()]
        self.out_edges = [arc for arc in self.outer_edges if arc[0] in self.subgrafo.nodes()]
        self.in_edges = [arc for arc in self.outer_edges if arc[1] in self.subgrafo.nodes()]

    def modularidad(self):
        q = 0
        if len(self.lista_nodos) > 1:
            for u in self.lista_nodos:
                for v in self.lista_nodos:
                    a = self.subgrafo.get_edge_data(u,v, default={'weight': 0})['weight']
                    b = self.supergrafo.in_degree(u)*self.supergrafo.out_degree(v)/(self.m)
                    q += 1/(self.m) * (a - b)
        return q

    def totals(self):
        return len(self.in_edges), len(self.out_edges)
    
    def sacar_nodo(self, nodo):
        if len(self.lista_nodos) == 1:
            self.lista_nodos = []
            self.subgrafo = self.supergrafo.subgraph(self.lista_nodos)
        else:
            self.lista_nodos.remove(nodo)
            self.subgrafo = self.supergrafo.subgraph(self.lista_nodos)
            self.outer_edges = [arc for arc in self.supergrafo.edges() if arc not in self.subgrafo.edges()]
            self.out_edges = [arc for arc in self.outer_edges if arc[0] in self.subgrafo.nodes()]
            self.in_edges = [arc for arc in self.outer_edges if arc[1] in self.subgrafo.nodes()]
            
    def agregar_nodo(self, nodo):
        self.lista_nodos.append(nodo)
        self.subgrafo = self.supergrafo.subgraph(self.lista_nodos)
        self.outer_edges = [arc for arc in self.supergrafo.edges() if arc not in self.subgrafo.edges()]
        self.out_edges = [arc for arc in self.outer_edges if arc[0] in self.subgrafo.nodes()]
        self.in_edges = [arc for arc in self.outer_edges if arc[1] in self.subgrafo.nodes()]

    def __repr__(self):
        return str(self.lista_nodos[0:10])


In [185]:


class Comunidades:
    def __init__(self,grafo: nx.Graph, lista_nodos,semilla=17):
        # parametros
        random.seed(semilla)
        self.lista_nodos = sorted(nodos, key=lambda x: random.randint(0, len(lista_nodos)))
        self.G = grafo
        self.comunidades = []
        self.m = self.G.size(weight = 'weight')
        
        # asignacion inicial
        for nodo in self.lista_nodos:
            self.comunidades.append(Comunidad(nodo, grafo))
        print(f'Número comunidades: {len(self.comunidades)}')

        # calculo de modularidad inicial
        Q = 0
        for com in self.comunidades:
            Q += com.modularidad()
        print(f'Modularidad inicial: {Q}')

    def run(self):
        for i in range(10):

            
            self.lista_nodos = sorted(self.lista_nodos, key=lambda x: random.randint(0, len(nodos)))
            nodo = self.lista_nodos[i]
            deltas = []
            for com in self.comunidades:
                deltas.append(self.delta_modularidad(nodo, com))
            k_max = np.argmax(deltas)
            print(f'delta:{deltas[k_max]}')
            best_com = self.comunidades[k_max]
            old_com = self.comunidades[self.comunidad_nodo(nodo)]
            best_com.agregar_nodo(nodo)
            #print(best_com)
            old_com.sacar_nodo(nodo)
            #print(old_com)
            if old_com.lista_nodos == []:
                self.comunidades.remove(old_com)
                
            Q = 0
            for com in self.comunidades:
                Q += com.modularidad()
            print(f'Modularidad: {Q}')
            #print(f'Número comunidades: {len(self.comunidades)}')
        
        

                    

    def delta_modularidad(self, nodo, com: Comunidad):
        if nodo in com.lista_nodos:
            return 0
        else:
            G = self.G.subgraph(com.lista_nodos + [nodo])
            a = G.degree(nodo)/self.m
            in_, out_ = com.totals()
            b  = (self.G.out_degree(nodo)*in_ + self.G.in_degree(nodo)*out_)/(self.m**2)
            return a - b
        
    def comunidad_nodo(self, nodo):
        for com in self.comunidades:
            if nodo in com.lista_nodos:
                return self.comunidades.index(com)
        print('Error!')




In [186]:
comunidades = Comunidades(grafo=Gd,lista_nodos=Gd.nodes)
comunidades.run()


Número comunidades: 77
Modularidad inicial: 0
delta:0.0012135633551457465
Modularidad: 0.003646638905413444
delta:0.0011972040452111839
Modularidad: 0.006041046995835811
delta:0.0011942296252230814
Modularidad: 0.010870017846519926
delta:0.001143664485425342
Modularidad: 0.013161808447352766
delta:0.001216537775133849
Modularidad: 0.011945270672218917
delta:0.0012046400951814396
Modularidad: 0.013149910767400356
delta:0.001143664485425342
Modularidad: 0.011943783462224866
delta:0.001216537775133849
Modularidad: 0.015596371207614514
delta:0.0012105889351576443
Modularidad: 0.018019036287923854
delta:0.001143664485425342
Modularidad: 0.019225163593099344
delta:0.0011942296252230814
Modularidad: 0.01800565139797739
delta:0.001188280785246877
Modularidad: 0.020371802498512788
delta:0.0011853063652587745
Modularidad: 0.021573468173706128
delta:0.001152587745389649
Modularidad: 0.021585365853658535
delta:0.0011659726353361094
Modularidad: 0.020295954788816177
delta:0.0012091017251635932
Modu